In [27]:
from __future__ import annotations
from dataclasses import dataclass

from typing import List
from typing import Callable


@dataclass
class Estado:
    nodo: List
    pai: Estado
    acao: str
    custo: float

    def __eq__(self, outro):
        if not isinstance(outro, Estado):
            return False

        return self.nodo == outro.nodo

    def __hash__(self):
        return hash(frozenset(self.nodo))   

@dataclass
class Problema:
    estado_inicial: Estado
    acoes: List[str]
    funcao_sucessora: Callable[[Estado], List[Estado]]
    funcao_objetivo: Callable[[Estado], bool] 

    def solucao(self, estado: Estado) -> List[Estado]:
        lista = [estado]

        while estado.pai != None:
            estado = estado.pai
            lista.append(estado)

        lista.reverse()
        
        return lista

In [28]:
from pprint import pprint

def busca(problema: Problema) -> List:
    """
    Implementação do algoritmo de busca.

    :param problema: definição do problema
    """

    # Inicialmente somente o estado inicial está na fronteira
    fronteira = [problema.estado_inicial]

    while True:

        pprint('fronteira: ')
        pprint([e.nodo for e in fronteira])
        print()

        # Verifica se a fronteira está vazia
        if not fronteira:
            raise RuntimeError('Nenhuma solução encontrada')

        # Remove o próximo elemento da fronteira
        #estado = fronteira.pop(0)     # FIFO - busca em largura
        #estado = fronteira.pop()      # LIFO - busca em profundidade

        #fronteira.sort(key=lambda e: e.custo)
        estado = fronteira.pop(0)     # HEAP - busca de custo uniforme

        pprint('Estado visitado: ')
        pprint(estado.nodo)
        pprint(f'Custo do caminho: {estado.custo}')
        print()

        # Testa o objetivo
        if problema.funcao_objetivo(estado):
            return problema.solucao(estado)

        # Expande o estado atual e adiciona os vizinhos na fronteira
        fronteira += problema.funcao_sucessora(estado)

In [29]:
"""
# Modelagem do problema

Estado:
    
    - Os estados representa cada um do lado da ponte
    - Cada pessoa é representado por um tempo indicando o tempo gasto para atravesar a ponte 

      LADO INICIAL    LADO FINAL
       A  B  C  D         P    
    [ [1, 2, 5, 8],      [ ]]

"""

acoes = {
    'A B -> P': (0, 1),     #ATRAVESANDO
    'P -> A': (1, 0),       #VOLTANDO
    'P -> B': (1, 0),       #VOLTANDO
    'A C -> P': (0, 1),     #ATRAVESANDO
    'P -> C': (1, 0),       #VOLTANDO
    'A D -> P': (0,1),      #ATRAVESANDO
    'P -> D': (1, 0),       #VOLTANDO
    'B C -> P': (0, 1),     #ATRAVESANDO
    'B D -> P': (0, 1),     #ATRAVESANDO
    'C D -> P': (0, 1)      #ATRAVESANDO
    }

visitado = []

def expansao(estado: Estado) -> List[Estado]:
    vizinhanca = []
    nodo = estado.nodo

    def copia_torres():
        return [nodo[0].copy(), nodo[1].copy()]

    for acao, movimento in acoes.items():
        de, para = movimento

        if nodo[de] and (not nodo[para] or nodo[de][-1] < nodo[para][-1]):
            torres = copia_torres()
            torres[para].append(torres[de].pop())
            
            novo_estado = Estado(
                nodo = torres,
                pai = estado,
                acao = acao,
                custo = estado.custo + 1,
            )

            # Expande o no somente se ele ainda não foi visitado
            if novo_estado not in visitado:
                vizinhanca.append(novo_estado)
                visitado.append(novo_estado)

    return vizinhanca


def teste_objetivo(estado: Estado):
    return estado.nodo == [[], [5, 8, 2, 1]]


problema = Problema(
    estado_inicial = Estado(
                        nodo = [[1, 2, 5, 8], []],
                        pai = None,
                        custo = 0,
                        acao = None,
                    ),
    acoes=['A B -> P', 'P -> A', 
           'P -> B', 'A C -> P',
           'P -> C', 'A D -> P',
           'P -> D', 'B C -> P',
           'B D -> P', 'C D -> P'],
    funcao_sucessora=expansao,
    funcao_objetivo=teste_objetivo,
)

solucao = busca(problema)

'fronteira: '
[[[1, 2, 5, 8], []]]

'Estado visitado: '
[[1, 2, 5, 8], []]
'Custo do caminho: 0'

'fronteira: '
[[[1, 2, 5], [8]]]

'Estado visitado: '
[[1, 2, 5], [8]]
'Custo do caminho: 1'

'fronteira: '
[[[1, 2], [8, 5]]]

'Estado visitado: '
[[1, 2], [8, 5]]
'Custo do caminho: 2'

'fronteira: '
[[[1], [8, 5, 2]]]

'Estado visitado: '
[[1], [8, 5, 2]]
'Custo do caminho: 3'

'fronteira: '
[[[], [8, 5, 2, 1]]]

'Estado visitado: '
[[], [8, 5, 2, 1]]
'Custo do caminho: 4'

'fronteira: '
[]



RuntimeError: ignored